## Training a Neural Tree

So far we've learned the basic structure of a `NeuralTree` and seen how task objects are used to interface with datasets.
Now we'll see how a `NeuralTree` is trained.


### Load the configuration file

In [ ]:
from omegaconf import OmegaConf
import hydra

with hydra.initialize(config_path="./hydra"):
    cfg = hydra.compose(config_name="3_training_a_neural_tree")
    OmegaConf.set_struct(cfg, False)

### Setup

In [ ]:
import lightning as L

# set random seed
L.seed_everything(seed=cfg.random_seed, workers=True)

# instantiate model
model = hydra.utils.instantiate(cfg.tree)
model.build_tree(cfg, skip_task_setup=False)

# instantiate trainer, set logger
trainer = hydra.utils.instantiate(cfg.trainer)

In [ ]:
trainer.fit(
    model,
    train_dataloaders=model.get_dataloader(split="train"),
    val_dataloaders=model.get_dataloader(split="val"),
)